# Run Qwen PPO with [verl](https://github.com/volcengine/verl)

This tutorial provides a step-by-step guide to using veRL for executing your RLHF pipeline. You can find our [github repo](https://github.com/volcengine/verl/) and [documentation](https://verl.readthedocs.io/en/latest/index.html) for mode details.

This notebook is also published on the [Lightning Studio](https://lightning.ai/hlin-verl/studios/verl-getting-started) platform, which provides free GPU quota every month. Checkout the published notebook with pre-installed dependencies using a free L4 GPU [here](https://lightning.ai/hlin-verl/studios/verl-getting-started) (no credit card required).

### You will learn:

- How to install veRL from scratch.
- How to use existing scripts to run an RLHF pipeline with your own models and data.

# Dependency Installation

If you are running on Lightning Studio using the published notebook, the dependencies are **already installed** and you can proceed to step "**Load Pretrained Language Model**"

In [2]:
!pip3 install --upgrade pip setuptools wheel
!pip3 install torch==2.4.0 torchvision==0.19.0
!pip3 list | grep torch
!pip3 install flash-attn --no-build-isolation

  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.19.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2

## Install and verify verl
Now we're ready to install verl!

In [4]:
# In case you run this notebook and have not cloned verl yet:
# !git clone https://github.com/volcengine/verl $HOME/verl_repo

!cd $HOME/projects/verl && pip3 install -e . -U

Obtaining file:///home/awhe/projects/verl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 31.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 MB 44.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 55.1 MB/s eta 0:00:00
  Building editable for verl (pyproject.toml) ... done
  Created wheel for verl: filename=verl-0.1-0.editable-py3-none-any.whl size=12887 sha256=3878b00314377e2938f11f26ca46a094c8ec29d670c9a6eb5a80627fbe295988
  Stored in directory: /tmp/pip-ephem-wheel-cache-ltaidwck/wheels/3a/a8/b4/31181b2bb49243306614a0473616d6f97d245420236f9a0149
Successfully built verl
  Attempting uninstall: verl
    Found existing

## Restart the python kernel

In [3]:
import IPython

# Restart the kernel to pickup the latest python packages
IPython.get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
import torch
try:
  assert torch.cuda.is_available() is True
  torch.ones(1, dtype=torch.bfloat16).cuda()
except AssertionError:
  print("Please switch to an env with GPUs supporting bfloat16 (L4 RTX 5000, A5000, A100, H100, A10, etc)")

try:
  import verl
except Exception as e:
  print("Please install verl via pip and restart the kernel")
  raise e

import flash_attn

/home/awhe/miniconda3/envs/verl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-01 17:00:16,772	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# Load Pretrained Language Model

verl supports models available in Huggingface transformers (as well as custom Megatron models).

Let's download the model first.

In [2]:
!huggingface-cli download Qwen/Qwen2.5-0.5B-Instruct --local-dir /data/user_data/awhe/hf/models/Qwen2.5-0.5B-Instruct

# If huggingface-cli is not stable, use the method below
# import transformers
# transformers.pipeline('text-generation', model='Qwen/Qwen2.5-0.5B-Instruct')

#/data/user_data/awhe/models/Qwen2.5-0.5B-Instruct

Traceback (most recent call last):
  File "/home/awhe/miniconda3/envs/verl/lib/python3.10/site-packages/huggingface_hub/utils/_http.py", line 406, in hf_raise_for_status
    response.raise_for_status()
  File "/home/awhe/miniconda3/envs/verl/lib/python3.10/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/Qwen/Qwen2.5-0.5B-Instruct/revision/main

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/awhe/miniconda3/envs/verl/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/home/awhe/miniconda3/envs/verl/lib/python3.10/site-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    service.run()
  File "/home/awhe/miniconda3/envs/verl/lib/python3.10/site-packages/huggingface_hub/commands/download.py", line 153, in 

# Dataset preparation

We train with the Grade School Math 8K (GSM8k) task in this demo. The dataset is downloaded from huggingface [gsm8k](https://huggingface.co/datasets/openai/gsm8k) and below are some samples:


**Prompt**

Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 teaspoons of sugar and cups of water, calculate the number of teaspoonfuls of sugar she used.

**Solution**

The total ratio representing the ingredients she used to make the coffee is 7+13 = <<7+13=20>>20 Since the fraction representing the number of teaspoons she used is 7/20, she used 7/20120 = <<7/20120=42>>42 #### 42

In [3]:
!mkdir -p /data/user_data/awhe/data/gsm8k
!python3 $HOME/projects/verl/examples/data_preprocess/gsm8k.py --local_dir /data/user_data/awhe/data/gsm8k

Using the latest cached version of the dataset since openai/gsm8k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'main' at /data/user_data/awhe/huggingface/datasets/openai___gsm8k/main/0.0.0/e53f048856ff4f594e959d75785d2c2d37b678ee (last modified on Wed Nov 13 23:28:26 2024).
Creating parquet from Arrow format: 100%|████████| 2/2 [00:00<00:00, 359.64ba/s]


# the reward

We use a rule-based reward model. We force the model to produce a final answer following 4 `#` as shown in the solution. We extract the final answer from both the solution and model's output using regular expression matching. We compare them and assign a reward of 1 to correct answer, 0.1 to incorrect answer and 0 to no answer.

In [4]:
import inspect
from verl.utils.reward_score.gsm8k import compute_score as gsm8k_reward
print(inspect.getsource(gsm8k_reward))

def compute_score(solution_str, ground_truth, method='strict', format_score=0., score=1.):
    """The scoring function for GSM8k.

    Reference: Trung, Luong, et al. "Reft: Reasoning with reinforced fine-tuning." Proceedings of the 62nd Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). 2024.

    Args:
        solution_str: the solution text
        ground_truth: the ground truth
        method: the method to extract the solution, choices are 'strict' and 'flexible'
        format_score: the score for the format
        score: the score for the correct answer
    """
    answer = extract_solution(solution_str=solution_str, method=method)
    if answer is None:
        return 0
    else:
        if answer == ground_truth:
            return score
        else:
            return format_score



# Run the RL Pipeline
Let's start with the Proximal Policy Optimization (PPO) algorithm,  one of the most widely used methods for post-training large language models.

The main entry point of the PPO algorithm example is: `main_ppo.py`. A detailed guide to understanding the code architecture of `main_ppo.py` is available [here](https://verl.readthedocs.io/en/latest/examples/ppo_code_architecture.html).

In this tutorial, we will demonstrate how to run the PPO algorithm with **Qwen 2.5-0.5B** by setting:
- `trainer.n_gpus_per_node`: Number of GPUs per node.

- `actor_rollout_ref.rollout.tensor_model_parallel_size`: TP size for rollout. Only effective for vllm.

- `actor_rollout_ref/critic.model.path`: Huggingface model path. This can be either local path or HDFS path. For HDFS path, we provide utils to download it to DRAM and convert the HDFS path to local path.

- `data.train_batch_size`: Batch size sampled for one training iteration of different RL algorithms.

- `data.max_prompt_length`: Maximum prompt length. All prompts will be left-padded to this length. An error will be reported if the length is too long.

- `data.max_response_length`: Maximum response length. Rollout in RL algorithms (e.g. PPO) generates up to this length.

- `actor_rollout_ref.actor.ppo_mini_batch_size`: One sample is split into multiple sub-batches with batch_size=ppo_mini_batch_size for PPO updates.

- `actor_rollout_ref/critic.actor.ppo_micro_batch_size`: Similar to gradient accumulation, the micro_batch_size for one forward pass, trading speed for GPU memory.

The full configuration explanation is available [here](https://verl.readthedocs.io/en/latest/examples/config.html).

The training may take a few hours to finish but you can observe how the model performance increases. It will progressively output:

- generated sentences.

- step information with RL metrics, such as entropy loss, kl, and ``val/test_score/openai/gsm8k`` (validated every ``trainer.test_freq`` steps)

If you come across GPU out of memory issues, set smaller values for the micro batch size used for gradient accumulation:

- actor_rollout_ref.actor.ppo_micro_batch_size=1
- critic.ppo_micro_batch_size=1

In [6]:
!PYTHONUNBUFFERED=1 python3 -m verl.trainer.main_ppo \
 data.train_files=/data/user_data/awhe/data/gsm8k/train.parquet \
 data.val_files=/data/user_data/awhe/data/gsm8k/test.parquet \
 data.train_batch_size=256 \
 data.val_batch_size=1312 \
 data.max_prompt_length=512 \
 data.max_response_length=256 \
 actor_rollout_ref.model.path=/data/user_data/awhe/models/Qwen2.5-0.5B-Instruct \
 actor_rollout_ref.actor.optim.lr=1e-6 \
 actor_rollout_ref.actor.ppo_mini_batch_size=64 \
 actor_rollout_ref.actor.ppo_micro_batch_size=1 \
 actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=1 \
 actor_rollout_ref.rollout.tensor_model_parallel_size=1 \
 actor_rollout_ref.rollout.gpu_memory_utilization=0.4 \
 actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu=4 \
 critic.optim.lr=1e-5 \
 critic.model.path=/data/user_data/awhe/models/Qwen2.5-0.5B-Instruct \
 critic.ppo_micro_batch_size=1 \
 algorithm.kl_ctrl.kl_coef=0.001 \
 +trainer.val_before_train=False \
 trainer.default_hdfs_dir=null \
 trainer.n_gpus_per_node=1 \
 trainer.nnodes=1 \
 trainer.save_freq=10 \
 trainer.test_freq=10 \
 trainer.total_epochs=15 \
 trainer.logger=\[console\]

2025-02-01 17:10:58,531	INFO worker.py:1841 -- Started a local Ray instance.
(main_task pid=3047267) {'actor_rollout_ref': {'actor': {'clip_ratio': 0.2,
(main_task pid=3047267)                                  'entropy_coeff': 0.001,
(main_task pid=3047267)                                  'fsdp_config': {'fsdp_size': -1,
(main_task pid=3047267)                                                  'grad_offload': False,
(main_task pid=3047267)                                                  'optimizer_offload': False,
(main_task pid=3047267)                                                  'param_offload': False,
(main_task pid=3047267)                                                  'wrap_policy': {'min_num_params': 0}},
(main_task pid=3047267)                                  'grad_clip': 1.0,
(main_task pid=3047267)                                  'kl_loss_coef': 0.001,
(main_task pid=3047267)                                  'kl_loss_type': 'low_var_kl',
(main_task pid=3047267)     

# Stop and clean up resources

In [12]:
!ray stop

Did not find any active Ray processes.
